In [2]:
import pandas as pd

In [22]:
import pandas as pd
import json
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Video_Games_5.json.gz')


In [4]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,NaN,NaN,NaN
1,4.0,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,NaN,NaN,NaN
2,3.0,True,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,NaN,NaN,NaN
3,2.0,True,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what...",Two Stars,1424390400,NaN,NaN,NaN
4,5.0,True,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since...",love this game,1419465600,NaN,NaN,NaN


In [5]:
df.shape

(497577, 12)

In [36]:
over3 = df[df['overall'] > 4]
under3 = df[df['overall'] < 2]
over3.head()
over3.shape

(299759, 12)

In [37]:
subset_size = 20000

# Randomly subset the data
over3 = over3.sample(n=subset_size, random_state=42)
under3 = under3.sample(n=subset_size,random_state=42)

In [38]:
under3.shape

(20000, 12)

In [39]:
over3.shape

(20000, 12)

In [40]:
# filter headlines
# for loop to filter dataset by keywords

keyword_list = ['bad','like','love','game']

#create an empty dataframe called result 
#for each keyword in my keyword list, if keyword is in text
#then we append this row to result 



result = pd.DataFrame() 

for index, row in under3.iterrows():
    if pd.isnull(row['reviewText']):
        continue
    text = row['reviewText']
    for keyword in keyword_list:
        if keyword in text:
            result = pd.concat([result, row.to_frame().T], ignore_index=True)
        else:
            continue


In [41]:
result.drop_duplicates(subset='reviewText', keep='first', inplace=True)

In [42]:
print(len(result))
result.head()

13428


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,False,"10 22, 2008",A74TA8X5YQ7NE,B000X9FV5M,NeuroSplicer,Recently UBISOFT had to settle a huge class-ac...,THE DAY GAMING CRIED...,1224633600,NaN,{'Format:': ' Video Game'},NaN
1,1.0,False,"03 23, 2012",A1JGSUX59ZTWYE,B004UDLRMS,Bear,well let me start by saying that when i first ...,last gen game,1332460800,6,{'Format:': ' Video Game'},NaN
4,1.0,False,"06 9, 2008",A1MRPX3RM48T2I,B000NV8F58,Trenton Bennett,"UPDATE, 2010: This review is pretty old, and ...",Never Again,1212969600,58,{'Format:': ' Video Game'},NaN
5,1.0,False,"07 4, 2014",A3DE438TF1A958,B002I0GKA4,thomas henry,What in the hell is this i can say that some o...,this game shouldnt be named resident evil,1404432000,NaN,{'Format:': ' Video Game'},NaN
7,1.0,False,"03 23, 2014",A3E9Y6IBUXV41U,B00GGUU71A,anonymous,i need to remind myself not to get dlc or seas...,uninspiring dlc maps,1395532800,NaN,NaN,NaN


In [43]:
import re
from sklearn import feature_extraction 
stop_words = feature_extraction.text.ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

def preprocess(text):
  text = text.lower() #lowercase
  text = re.sub(r'[^\w\s]', '', text) #remove punctuations
  text = re.sub(r'\d+', '', text) #remove numbers
  text = " ".join(text.split()) #stripWhitespace
  text = text.split()
  l_keyword_list = [element.lower() for element in keyword_list]
  new_stop_words = frozenset(stop_words.union(l_keyword_list)) 
  text = [x for x in text if x not in new_stop_words] #remove stopwords
  text = [x for x in text if x not in ["game", "gaming"]] #remove task specific stopwords
  text = " ".join(text)
  # stemmer_ps = PorterStemmer()  
  # text = [stemmer_ps.stem(word) for word in text.split()] #stemming
  # text = " ".join(text)
  # lemmatizer = WordNetLemmatizer()
  # text = [lemmatizer.lemmatize(word) for word in text.split()]  #lemmatization
  # text = " ".join(text)
  
  return(text)

In [44]:
under3['text_processed']=under3['reviewText'].apply(lambda x:preprocess(str(x)))
under3['text_processed']=under3['text_processed'].apply(lambda x:x.split())

In [45]:
from gensim import corpora
dictionary = corpora.Dictionary(under3['text_processed'])
dictionaryDF = pd.DataFrame()
dictionaryDF['id']=dictionary.keys()
dictionaryDF['word']=dictionary.values()
dictionaryDF

,id,word
0,0,accentuated
1,1,access
2,2,activations
3,3,apparently
4,4,artform
...,...,...
46217,46217,advertiser
46218,46218,controlsdull
46219,46219,gameplayterrible
46220,46220,outcrap


In [46]:
dictionary.filter_extremes(no_below=100, no_above=100000)
# no_below (int, optional) – Keep tokens which are contained in at least no_below documents.
# no_above (float, optional) – Keep tokens which are contained in no more than no_above documents (fraction of total corpus size, not an absolute number).
dictionaryDF = pd.DataFrame()
dictionaryDF['id']=dictionary.keys()
dictionaryDF['word']=dictionary.values()
dictionaryDF

,id,word
0,0,access
1,1,apparently
2,2,attempt
3,3,battle
4,4,begin
...,...,...
1502,1502,locked
1503,1503,grind
1504,1504,vita
1505,1505,videos


In [47]:
dictionary = corpora.Dictionary(under3['text_processed'])
under3['text_ids']=under3['text_processed'].apply(lambda x:dictionary.doc2bow(x))

In [48]:
from gensim import models
num_topics=10
ldamodel = models.ldamodel.LdaModel(under3['text_ids'], num_topics = num_topics, id2word=dictionary, passes=1, random_state=123)
topics = ldamodel.print_topics(num_words=8)
for topic in topics:
    print(topic)

(0, '0.010*"story" + 0.008*"just" + 0.008*"graphics" + 0.007*"good" + 0.007*"characters" + 0.007*"character" + 0.006*"level" + 0.006*"play"')
(1, '0.019*"product" + 0.016*"work" + 0.011*"headset" + 0.010*"quality" + 0.010*"use" + 0.009*"working" + 0.009*"bought" + 0.008*"buy"')
(2, '0.015*"just" + 0.009*"time" + 0.008*"games" + 0.007*"really" + 0.006*"dont" + 0.006*"good" + 0.006*"fun" + 0.006*"graphics"')
(3, '0.016*"play" + 0.013*"games" + 0.013*"just" + 0.009*"dont" + 0.009*"time" + 0.007*"money" + 0.006*"online" + 0.006*"really"')
(4, '0.034*"xbox" + 0.033*"controller" + 0.026*"ps" + 0.022*"console" + 0.012*"games" + 0.012*"halo" + 0.009*"just" + 0.009*"microsoft"')
(5, '0.014*"just" + 0.013*"games" + 0.011*"play" + 0.008*"ps" + 0.008*"dont" + 0.007*"wii" + 0.006*"work" + 0.006*"buy"')
(6, '0.009*"screen" + 0.008*"just" + 0.006*"button" + 0.006*"use" + 0.005*"remote" + 0.005*"right" + 0.005*"controller" + 0.005*"stick"')
(7, '0.014*"play" + 0.012*"games" + 0.012*"ea" + 0.011*"work"

In [16]:
from gensim import models
ldamodel.save('model.lda')
ldamodel =  models.LdaModel.load('model.lda')

In [17]:
pd.DataFrame(ldamodel.get_topics(), columns=ldamodel.id2word.values(), index=[f'topic {i}' for i in range(ldamodel.num_topics)])


,accentuated,access,activations,apparently,artform,attempt,backdoor,battle,begin,best,...,funfunny,boomerang,jungles,sia,cackles,crow,hag,ominous,piercings,pig
topic 0,0.000002,0.000084,0.000017,0.000385,0.000002,0.000421,0.000016,0.000700,0.000284,0.001393,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
topic 1,0.000009,0.000231,0.000012,0.000328,0.000011,0.000102,0.000023,0.000095,0.000206,0.000940,...,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
topic 2,0.000023,0.000367,0.000050,0.000247,0.000024,0.000317,0.000033,0.000484,0.000080,0.001157,...,0.000017,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013
topic 3,0.000007,0.000214,0.000017,0.000272,0.000007,0.000399,0.000093,0.000279,0.000067,0.001056,...,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004
topic 4,0.000013,0.000134,0.000040,0.000076,0.000024,0.000119,0.000014,0.000260,0.000277,0.001338,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
topic 5,0.000001,0.000571,0.000027,0.000544,0.000002,0.000355,0.000002,0.000425,0.000118,0.001580,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
topic 6,0.000007,0.000306,0.000012,0.000226,0.000008,0.000156,0.000009,0.000938,0.000255,0.001144,...,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
topic 7,0.000016,0.000051,0.000017,0.000151,0.000016,0.000099,0.000019,0.000577,0.000098,0.000879,...,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012
topic 8,0.000016,0.000070,0.000021,0.000434,0.000018,0.000152,0.000020,0.000343,0.000098,0.001656,...,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014,0.000014
topic 9,0.000007,0.000349,0.000020,0.000199,0.000011,0.000290,0.000008,0.000609,0.000125,0.001999,...,0.000006,0.000008,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007


In [ ]:
subset_size = 50000

# Randomly subset the data
subset = df.sample(n=subset_size, random_state=42)

In [49]:
keyword_list = ['good','like','love']

#create an empty dataframe called result 
#for each keyword in my keyword list, if keyword is in text
#then we append this row to result 



result2 = pd.DataFrame() 

for index, row in over3.iterrows():
    if pd.isnull(row['reviewText']):
        continue
    text = row['reviewText']
    for keyword in keyword_list:
        if keyword in text:
            result2 = pd.concat([result2, row.to_frame().T], ignore_index=True)
        else:
            continue


In [50]:
result2.drop_duplicates(subset='reviewText', keep='first', inplace=True)

In [52]:
print(len(result2))
result2.head()

8352


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,False,"12 31, 2001",A3EKMV82I91UPJ,B00000K4MC,Danny Carroll,This is a good game to get into if you've got ...,Fun,1009756800,NaN,{'Platform:': ' PC'},NaN
2,5.0,True,"01 4, 2011",A387VRSI2OET79,B000MKA60W,Adrian Jordan,"I was a bit wary when I got this, as I don't l...",Very Awesome.,1294099200,NaN,{'Format:': ' Video Game'},NaN
3,5.0,True,"07 24, 2017",A2RHCIPSKHZK6M,B004AC6THC,Riardas Sidoraviius,love it.,Five Stars,1500854400,NaN,NaN,NaN
4,5.0,True,"12 2, 2012",A22A1B1E26OZ9T,B0096KENEO,Shelli Landers,it was fast getting to the house and it was in...,very good,1354406400,NaN,"{'Edition:': ' Standard', 'Platform for Displa...",NaN
5,5.0,False,"02 27, 2013",A1ZCO2VVV5LSTJ,B000B8J7JQ,Marshall WahlstromHelgren,I love this game what I think is so fun about ...,2005 Super Mario Strikers for Gamecube,1361923200,NaN,NaN,NaN


In [53]:
over3['text_processed']=over3['reviewText'].apply(lambda x:preprocess(str(x)))
over3['text_processed']=over3['text_processed'].apply(lambda x:x.split())

In [54]:
from gensim import corpora
dictionary = corpora.Dictionary(over3['text_processed'])
dictionaryDF = pd.DataFrame()
dictionaryDF['id']=dictionary.keys()
dictionaryDF['word']=dictionary.values()
dictionaryDF

,id,word
0,0,challenge
1,1,challenging
2,2,combination
3,3,crawlers
4,4,dungeon
...,...,...
41928,41928,exhaustively
41929,41929,hrefshinmegamitenseipersonadpbcgvirefcm_cr_arp...
41930,41930,doozy
41931,41931,probems


In [55]:
dictionary = corpora.Dictionary(over3['text_processed'])
over3['text_ids']=over3['text_processed'].apply(lambda x:dictionary.doc2bow(x))

In [57]:
from gensim import models
num_topics=10
ldamodel = models.ldamodel.LdaModel(over3['text_ids'], num_topics = num_topics, id2word=dictionary, passes=1, random_state=123)
topics = ldamodel.print_topics(num_words=8)
for topic in topics:
    print(topic)

(0, '0.011*"just" + 0.009*"games" + 0.009*"play" + 0.008*"really" + 0.007*"time" + 0.007*"story" + 0.007*"great" + 0.006*"fun"')
(1, '0.038*"games" + 0.023*"play" + 0.019*"fun" + 0.018*"great" + 0.015*"best" + 0.012*"mario" + 0.012*"played" + 0.010*"graphics"')
(2, '0.017*"dance" + 0.011*"thanks" + 0.010*"ok" + 0.009*"great" + 0.009*"just" + 0.009*"play" + 0.008*"buy" + 0.008*"songs"')
(3, '0.014*"really" + 0.013*"fun" + 0.008*"great" + 0.007*"batman" + 0.006*"play" + 0.006*"games" + 0.006*"playing" + 0.005*"souls"')
(4, '0.021*"sims" + 0.017*"nice" + 0.011*"best" + 0.011*"fit" + 0.011*"grip" + 0.010*"workout" + 0.009*"balance" + 0.009*"k"')
(5, '0.029*"great" + 0.014*"ps" + 0.013*"works" + 0.012*"controller" + 0.010*"games" + 0.010*"use" + 0.009*"xbox" + 0.008*"just"')
(6, '0.011*"just" + 0.010*"pokemon" + 0.009*"perfect" + 0.009*"thank" + 0.007*"play" + 0.006*"new" + 0.005*"daughter" + 0.005*"mode"')
(7, '0.007*"games" + 0.006*"time" + 0.005*"fun" + 0.005*"play" + 0.005*"enemies" + 0